In [1]:
from config import *
import requests
import pandas as pd
from chatbot import chat
from message_sender import sender

pd.set_option('display.max_columns',False)
pd.set_option('display.max_rows',False)

In [2]:
df=pd.read_excel(data_path) 

In [3]:
for i in df.iterrows():
    df['Phone']='+923210214646'

In [ ]:
import random
import time
from datetime import datetime

error_df=pd.DataFrame(columns=df.columns)
i = 0
rand = random.randint(1, 4)
limit = min(len(df), 25 + rand)

while i < limit:
    row = df.iloc[i]

    phone_no = str(row['Phone']).strip()
    name = row['Name']

    if not phone_no.startswith('+'):
        phone_no = '+' + phone_no

    user_prompt = str(row)

    try:
        message = chat(user_prompt)
    except Exception as e:
        print("LLM error:", e)
        error_df=pd.concat([error_df,row.to_frame().T],ignore_index=True)
        i += 1
        continue

    try:
        sender(phone_no, message)
        print(f"Message sent to {name} on {phone_no} at {datetime.now().strftime('%H:%M:%S')}")
    except Exception as e:
        print(f"WhatsApp error for {phone_no} ({name}):", e)
    time.sleep(1200+random.randint(1,300))
    i += 1
if not error_df.empty:
    error_df.to_excel("Missed_Rows.xlsx",index=False)
